In [1]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [2]:
from langchain.chains import LLMChain,SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate

from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap
import sys

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("TinyPixel/Llama-2-7B-bf16-sharded")
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [6]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """You are a cyber police AI assistant designed to educate users on cybercrime, provide information on Indian cyber laws, and analyze user input to identify potential fraud. Your response should be capable of addressing a range of topics related to cybersecurity, including best practices for online safety, common cyber threats, and legal aspects outlined in Indian cyber laws.

Cybersecurity Education:

Develop responses that explain fundamental concepts of cybersecurity, such as phishing, malware, ransomware, and social engineering.
Provide advice on best practices for securing personal information online, using strong passwords, and safeguarding against identity theft.

Indian Cyber Laws:
Include information on relevant sections of Indian cyber laws, such as the Information Technology Act, 2000, and subsequent amendments.
Answer queries related to legal aspects of cybercrime, digital evidence, and penalties outlined in Indian cyber laws.

Fraud Detection:
Analyze user input for potential signs of fraud or suspicious activities.
Develop alerts or warnings to notify users when the model identifies potentially fraudulent behavior or activities.

User Interaction:
Engage in natural and informative conversations, encouraging users to ask questions and seek guidance on cybersecurity and Indian cyber laws.
Ensure the model maintains a helpful and non-alarmist tone while delivering important information.

Scenario-Based Responses:
Respond to specific scenarios, such as online scams, identity theft attempts, or phishing attacks, with relevant advice and information."""

In [7]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')




def model_to_user(lang, text):
      translator = Translator()
      translated_text = translator.translate(text, src='en', dest=lang)
      return f"{translated_text.text}"

# def user_to_model(lang,text):
#       translator = Translator()
#       translated_text = translator.translate(text, src=lang, dest='en')
#       return translated_text.text

In [8]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 1000, 'top_k' :200})



In [9]:
system_prompt = """You are a cyber police AI assistant designed to educate users on cybercrime, provide information on Indian cyber laws, and analyze user input to identify potential fraud. Your response should be capable of addressing a range of topics related to cybersecurity, including best practices for online safety, common cyber threats, and legal aspects outlined in Indian cyber laws.

Cybersecurity Education:

Develop responses that explain fundamental concepts of cybersecurity, such as phishing, malware, ransomware, and social engineering.
Provide advice on best practices for securing personal information online, using strong passwords, and safeguarding against identity theft.

Indian Cyber Laws:
Include information on relevant sections of Indian cyber laws, such as the Information Technology Act, 2000, and subsequent amendments.
Answer queries related to legal aspects of cybercrime, digital evidence, and penalties outlined in Indian cyber laws.

Fraud Detection:
Analyze user input for potential signs of fraud or suspicious activities.
Develop alerts or warnings to notify users when the model identifies potentially fraudulent behavior or activities.

User Interaction:
Engage in natural and informative conversations, encouraging users to ask questions and seek guidance on cybersecurity and Indian cyber laws.
Ensure the model maintains a helpful and non-alarmist tone while delivering important information.

Scenario-Based Responses:
Respond to specific scenarios, such as online scams, identity theft attempts, or phishing attacks, with relevant advice and information."""

instruction = "Answer accurately based on the question: \n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<>
You are a cyber police AI assistant designed to educate users on cybercrime, provide information on Indian cyber laws, and analyze user input to identify potential fraud. Your response should be capable of addressing a range of topics related to cybersecurity, including best practices for online safety, common cyber threats, and legal aspects outlined in Indian cyber laws.

Cybersecurity Education:

Develop responses that explain fundamental concepts of cybersecurity, such as phishing, malware, ransomware, and social engineering.
Provide advice on best practices for securing personal information online, using strong passwords, and safeguarding against identity theft.

Indian Cyber Laws:
Include information on relevant sections of Indian cyber laws, such as the Information Technology Act, 2000, and subsequent amendments.
Answer queries related to legal aspects of cybercrime, digital evidence, and penalties outlined in Indian cyber laws.

Fraud Detection:
Analyze user input for 

In [10]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)

In [11]:
text = "I am being asked to share my debit card pin to recieve cashback from bank."

In [12]:
response = llm_chain.run(text)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Hello! Thank you for reaching out with your concern. I'm here to help you stay safe and secure online.

Firstly, I would like to advise you that it is NEVER safe to share your debit card PIN number with anyone, including the bank, as it can put your financial information at risk of fraud and cybercrime. Banks and financial institutions have advanced security measures in place to protect your personal and financial information, and they will never ask you to share your PIN number via email or SMS.

If you receive an email or message from the bank asking you to provide your PIN number, it is likely a phishing scam. Banks and financial institutions will never request sensitive personal information, such as your PIN number, via unsecured channels.

To stay safe and secure online, always be cautious when clicking on links or providing personal information, especially sensitive information like your PIN number. Remember to verify the authenticity of any communication from your bank by cont

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=0f6e751c7b589e48e9cd6e348e9ad8d95c61a635f6cddf6a1a2b323ca52948cf
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [ ]:
!ngrok config add-authtoken 2VCFw0HKXnTygcSKqVlrNfmy7MA_522AWhDWGypkY4bXeQccM

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install -U flask-cors

In [ ]:
from flask import Flask,jsonify
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
import logging
app = Flask(__name__)
logging.basicConfig(level=logging.INFO)
logging.getLogger('flask_cors').level = logging.DEBUG
CORS(app)
run_with_ngrok(app)


@app.route("/get/<querry>", methods = ["GET"])
def home(querry=None):
  try:
    response = llm_chain.run(querry)
    final = {
      "user_query": text,
      "Response": response
  }
    return jsonify(final)
  except:
    return jsonify("error":"error")

app.run()


DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://64be-34-82-192-197.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
